In [1]:
# import lib
import sys
import os
import pandas as pd
import argparse
from yolo import YOLO, detect_video
from PIL import Image

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [16]:
args = {'model': YOLO.get_defaults("model_path"),
        'anchors': YOLO.get_defaults("anchors_path"),
        'classes': YOLO.get_defaults("classes_path"),
        'gpu_num': YOLO.get_defaults("gpu_num"),
        'input': 'img_sample',
        'output': 'output',
        'score_file': 'submission.csv',
        'class_file': 'model_data/oi_classes.csv',
       }

In [4]:
yolo = YOLO(**args)
os.makedirs(yolo.output, exist_ok=True)

model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [26]:
# load Open Images classes
oi_class = pd.read_csv(args['class_file'])
display(oi_class)

,Label,Name
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football
...,...,...
596,/m/0qmmr,Wheelchair
597,/m/0wdt60w,Rugby ball
598,/m/0xfy,Armadillo
599,/m/0xzly,Maracas


,Label,Name
0,/m/011k07,Tortoise


In [50]:
df = pd.DataFrame(columns=['ImageID', 'PredictionString'])
for img_name in os.listdir(yolo.input):
    try:
        image = Image.open(os.path.join(yolo.input, img_name))
    except:
        print('Open Error!')
        print(img_name)
        continue
    else:
        out_boxes, out_scores, out_classes, r_image = yolo.detect_image(image)

    # save image
    r_image.save(os.path.join(yolo.output, img_name))

    pred_str = ""
    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = yolo.class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        
        # search class label
        mask = oi_class['Name'] == str.capitalize(predicted_class)
        labels = oi_class[mask]['Label'].values
        
        # labels is empty, class not found
        if(labels.size == 0):
            continue
            
        label = labels[0]
        top, left, bottom, right = box
        xmin = left / image.width
        xmax = right / image.width
        ymin = bottom / image.height
        ymax = top / image.height
        pred_str += " {} {} {} {} {} {}".format(label, score, xmin, ymin, xmax, ymax)

    # save score
    imageid = os.path.splitext(os.path.basename(img_name))[0]
    s = pd.Series([imageid, pred_str], index=df.columns)
    df = df.append(s, ignore_index=True)

df.to_csv(args['score_file'], index=False, header=True)

(416, 416, 3)
Found 9 boxes for img
handbag 0.49 (624, 441) (698, 545)
truck 0.36 (780, 159) (1011, 382)
truck 0.45 (898, 149) (1015, 413)
truck 0.99 (64, 80) (622, 584)
person 0.35 (615, 288) (698, 412)
person 0.91 (583, 289) (655, 410)
person 0.94 (759, 250) (876, 605)
person 0.98 (883, 210) (976, 571)
person 0.99 (454, 274) (549, 597)
1.1595149069999025
(416, 416, 3)
Found 22 boxes for img
book 0.56 (502, 532) (585, 590)
tvmonitor 0.80 (812, 296) (876, 372)
chair 0.38 (5, 529) (139, 675)
chair 0.48 (300, 415) (389, 519)
person 0.40 (54, 262) (657, 399)
person 0.50 (518, 317) (603, 427)
person 0.55 (503, 300) (552, 365)
person 0.57 (28, 289) (108, 370)
person 0.61 (1, 326) (62, 554)
person 0.68 (221, 309) (314, 429)
person 0.73 (166, 280) (212, 336)
person 0.73 (107, 284) (153, 338)
person 0.79 (208, 280) (253, 338)
person 0.81 (463, 403) (612, 552)
person 0.90 (622, 329) (775, 471)
person 0.93 (138, 368) (320, 656)
person 0.96 (479, 224) (533, 354)
person 0.96 (339, 301) (433, 415)


In [51]:
display(df)

,ImageID,PredictionString
0,00000b4dcff7f799,/m/080hkjn 0.4898366928100586 0.6091075539588...
1,00001a21632de752,/m/0bt_c3 0.559672474861145 0.489931493997573...
2,0000d67245642c5f,


In [ ]:
yolo.close_session()